In [39]:
from __future__ import division
from keras.models import load_model
import csv
import copy
import numpy as np
import distance
from jellyfish._jellyfish import damerau_levenshtein_distance
import unicodecsv
from sklearn import metrics
from math import sqrt
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import tensorflow as tf
import pm4py

In [15]:
# attributes that may differ based on data structure or settings
dataset_local_path = "/Users/sumyea/Desktop/Study/Study/SS23/MLAPM/Code/Inter_case_aware_RTM-master/data/"
model_local_path = "Models_production/"
result_local_path = "Results_production/"
equipment_attribute = 'Equipment ID'
caseid_attribute = 'case:concept:name'
caseid_attr = 'Order OID'
arrived_timestamp_attribute = 'Arrived Timestamp'

In [16]:
"""
load dataset from eventlog
"""
df = pd.read_csv("production_mapped.csv")

In [17]:
df = df[['Case ID', 'Activity', 'Complete Timestamp']]
df

,Case ID,Activity,Complete Timestamp
0,0,0,2012-01-02 01:15:00
1,1,1,2012-01-02 04:50:00
2,2,2,2012-01-02 07:00:00
3,3,3,2012-01-02 09:30:00
4,1,4,2012-01-02 11:00:00
...,...,...,...
4538,216,0,2012-03-30 11:50:00
4539,207,7,2012-03-30 12:00:00
4540,157,3,2012-03-30 12:46:00
4541,171,1,2012-03-31 01:00:00


In [18]:
"""
initialize variables
"""

lines = [] #these are all the activity sequence
case_ids = []
time_seqs = []  # relative time since previous event
time_seqs2 = [] # relative time since case start
time_seqs3 = [] # absolute time of previous event

#helper variables
last_case = ''
first_line = True
num_lines = 0
case_start_time = None
last_event_time = None
ascii_offset = 161

In [19]:
def create_activity_time_sequence(df:pd.DataFrame, lines, case_ids, time_seqs, time_seqs2, time_seqs3, last_case, first_line, num_lines, case_start_time, last_event_time):
    
    """
    Creates activity and time sequence from the data.

    Args:
        df: A dataframe containing the activity and timestamp
        lines: An empty list to save the acitivity sequence
        case_ids: An empty list to record the case ids
        time_seqs: An empty list to save the time sequences (relative time since previous event)
        time_seqs2: An empty list to save the time sequences (relative time since case start)
        time_seqs3: An empty list to record the time sequences (absolute time of previous event)
        last_case: A string to save the last executed case for the loop
        first_line: A boolean value to indicate the start of a sequence
        num_lines: An integer value to indicate number of lines
        case_start_time: A datetime object to represent start of a sequence
        last_event_time: A datetime object to represent the end time of last event

    Returns:
        `lines`, `time_seqs`, `time_seqs2`, `time_seqs3`, `num_lines`
    """
    line = ''
    times = []
    times2 = []
    times3 = []
    
    for index, row in df.iterrows():
        t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S") #creates a datetime object from row[2]
        #t = time.strptime(row[2], "%d/%m/%y %H:%M") #for toy
        if row[0]!=last_case:
            case_ids.append(row[0]) 
            case_start_time = t
            last_event_time = t
            last_case = row[0]
            if not first_line:
                lines.append(line)
                time_seqs.append(times)
                time_seqs2.append(times2)
                time_seqs3.append(times3)
            line = ''
            times = []
            times2 = []
            times3 = []
            num_lines+=1
        line+=chr(int(row[1])+ascii_offset)
        time_since_last_event = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(last_event_time))
        time_since_case_start = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(case_start_time))
        midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
        time_since_midnight = datetime.fromtimestamp(time.mktime(t))-midnight
        time_diff = 86400 * time_since_last_event.days + time_since_last_event.seconds
        time_diff2 = 86400 * time_since_case_start.days + time_since_case_start.seconds
        times.append(time_diff)
        times2.append(time_diff2)
        times3.append(datetime.fromtimestamp(time.mktime(t)))
        last_event_time = t
        first_line = False
    lines.append(line)
    time_seqs.append(times)
    time_seqs2.append(times2)
    time_seqs3.append(times3)
    num_lines+=1
    
    return lines, case_ids, time_seqs, time_seqs2, time_seqs3, num_lines

In [20]:
lines, case_ids, time_seqs, time_seqs2, time_seqs3, num_lines = create_activity_time_sequence(df, lines, case_ids, time_seqs, time_seqs2, time_seqs3, last_case, first_line, num_lines, case_start_time, last_event_time)

In [171]:
print(time_seqs)
#print(time_seqs2)

[[0, 660, 180, -780, 660], [0, 660, 60, 60, 180], [0, 660, 120, 180, 120]]


In [173]:
print(time_seqs3)

[[datetime.datetime(2023, 12, 4, 16, 1), datetime.datetime(2023, 12, 4, 16, 12), datetime.datetime(2023, 12, 4, 16, 15), datetime.datetime(2023, 12, 4, 16, 2), datetime.datetime(2023, 12, 4, 16, 13), datetime.datetime(2023, 12, 4, 16, 2), datetime.datetime(2023, 12, 4, 16, 13), datetime.datetime(2023, 12, 4, 16, 14), datetime.datetime(2023, 12, 4, 16, 15), datetime.datetime(2023, 12, 4, 16, 18), datetime.datetime(2023, 12, 4, 16, 2), datetime.datetime(2023, 12, 4, 16, 13), datetime.datetime(2023, 12, 4, 16, 15), datetime.datetime(2023, 12, 4, 16, 18), datetime.datetime(2023, 12, 4, 16, 20)], [datetime.datetime(2023, 12, 4, 16, 1), datetime.datetime(2023, 12, 4, 16, 12), datetime.datetime(2023, 12, 4, 16, 15), datetime.datetime(2023, 12, 4, 16, 2), datetime.datetime(2023, 12, 4, 16, 13), datetime.datetime(2023, 12, 4, 16, 2), datetime.datetime(2023, 12, 4, 16, 13), datetime.datetime(2023, 12, 4, 16, 14), datetime.datetime(2023, 12, 4, 16, 15), datetime.datetime(2023, 12, 4, 16, 18), dat

In [21]:
def get_divisors(time_seqs, time_seqs2):
    """
    Calculates average time between events and average time between current and first events
    """
    
    divisor = np.mean([item for sublist in time_seqs for item in sublist]) #average time between events
    print('divisor: {}'.format(divisor))
    divisor2 = np.mean([item for sublist in time_seqs2 for item in sublist]) #average time between current and first events
    print('divisor2: {}'.format(divisor2))
    divisor3 = np.mean(list(map(lambda x: np.mean(list(map(lambda y: x[len(x)-1]-y, x))), time_seqs2)))
    print('divisor3: {}'.format(divisor3))
    
    return divisor, divisor2, divisor3

In [22]:
divisor, divisor2, divisor3 = get_divisors(time_seqs, time_seqs2)

divisor: 130.84305524983492
divisor2: 147.86704820603126
divisor3: 69.35168485999051


In [23]:
"""
Create three different folds from the activity and time sequences. 
The first two fold for training the LSTM model, the thrid fold for prediction.
"""

def get_elements_per_fold(num_lines):
    return int(round(num_lines/3))

def create_fold(lines, case_ids, time_seqs, time_seqs2, time_seqs3, elems_per_fold, n):
    if(n*elems_per_fold < len(lines)):
        k = n*elems_per_fold
    else:
        k = len(lines)
    fold = lines[(n-1)*elems_per_fold:k]
    fold_c = case_ids[(n-1)*elems_per_fold:k]
    fold_t = time_seqs[(n-1)*elems_per_fold:k]
    fold_t2 = time_seqs2[(n-1)*elems_per_fold:k]
    fold_t3 = time_seqs3[(n-1)*elems_per_fold:k]
    
    return fold, fold_c, fold_t, fold_t2, fold_t3

In [24]:
elems_per_fold = get_elements_per_fold(num_lines)
fold1, fold1_c, fold1_t, fold1_t2, fold1_t3 = create_fold(lines, case_ids, time_seqs, time_seqs2, time_seqs3, elems_per_fold, 1)
fold2, fold2_c, fold2_t, fold2_t2, fold2_t3 = create_fold(lines, case_ids, time_seqs, time_seqs2, time_seqs3, elems_per_fold, 2)
fold3, fold3_c, fold3_t, fold3_t2, fold3_t3 = create_fold(lines, case_ids, time_seqs, time_seqs2, time_seqs3, elems_per_fold, 3)

lines = fold1 + fold2
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2

In [25]:
step = 1
softness = 0
lines = list(map(lambda x: x+'!',lines)) #put delimiter symbol
max_len = max(map(lambda x: len(x),lines)) #find maximum line size

def get_total_and_target_chars(lines):
    """
    This function finds out all possible characters for events and annotates them with numbers,
    finds out all target characters for prediction,
    creates four different dictionaries from total characters and target characters.
    """
    
    total_chars = map(lambda x: set(x),lines)
    total_chars = list(set().union(*total_chars))
    total_chars.sort()
    target_chars = copy.copy(total_chars)
    total_chars.remove('!')
    print('total chars: {}, target chars: {}'.format(len(total_chars), len(target_chars)))
    char_indices = dict((c, i) for i, c in enumerate(total_chars))
    indices_char = dict((i, c) for i, c in enumerate(total_chars))
    target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
    target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
    
    return total_chars, target_chars, char_indices, indices_char, target_char_indices, target_indices_char

In [26]:
total_chars, target_chars, char_indices, indices_char, target_char_indices, target_indices_char = get_total_and_target_chars(lines)

total chars: 28, target chars: 29


In [27]:
lines = fold1+fold2+fold3
case_ids = fold1_c + fold2_c+fold3_c
lines_t = fold1_t+fold2_t+fold3_t
lines_t2 = fold1_t2+fold2_t2+fold3_t2
lines_t3 = fold1_t3+fold2_t3+fold3_t3

In [29]:
# load model, set this to the model generated by train.py
# choose the model which was generated on the last epoch
model = load_model(model_local_path+'model_16-1.40.h5')

Metal device set to: Apple M1


2023-06-25 19:16:06.650354: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-25 19:16:06.651931: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [30]:
def encode(sentence, times, times3, max_len=max_len):
    '''
    This function encodes the input for the trained LSTM model 
    '''
    
    num_features = len(total_chars)+5
    X = np.zeros((1, max_len, num_features), dtype=np.float32)
    leftpad = max_len-len(sentence)
    times2 = np.cumsum(times)
    for t, char in enumerate(sentence):
        midnight = times3[t].replace(hour=0, minute=0, second=0, microsecond=0)
        time_since_midnight = times3[t]-midnight
        multiset_abstraction = Counter(sentence[:t+1])
        for c in total_chars:
            if c==char:
                X[0, t+leftpad, char_indices[c]] = 1
        X[0, t+leftpad, len(total_chars)] = t+1
        X[0, t+leftpad, len(total_chars)+1] = times[t]/divisor
        X[0, t+leftpad, len(total_chars)+2] = times2[t]/divisor2
        #print(times[t]/divisor, times2[t]/divisor2, time_since_midnight.seconds/86400, times3[t].weekday()/7)
        X[0, t+leftpad, len(total_chars)+3] = time_since_midnight.seconds/86400
        X[0, t+leftpad, len(total_chars)+4] = times3[t].weekday()/7
        #print(times3[t], midnight)
    return X

In [31]:
def getSymbol(predictions):

    maxPrediction = 0
    maxindice = 0
    symbol = ''
    i = 0;
    for prediction in predictions:
        if(prediction>=maxPrediction):
            maxPrediction = prediction
            symbol = target_indices_char[i]
            maxindice = i
        i += 1
    return symbol, maxPrediction

In [30]:
print(len(lines_t3[0]), len(lines[0]))

2 2


In [32]:
# make predictions with predicted as ground_truth

predict_size = 1
def make_prediction(lines, case_ids, lines_t, lines_t3):
    with open(result_local_path+'result.csv', 'w') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(["case:concept:name", "Prefix length", "Groud truth", "Predicted", "Levenshtein", "Ground truth times", "Predicted times", "MAE"])
        for line, caseid, times, times3 in zip(lines, case_ids, lines_t, lines_t3):
            #print(prefix_size)
            
            for prefix_size in range(2, max_len):
                times.append(0)
                if(prefix_size==2):
                    cropped_line = ''.join(line[:prefix_size])
                    cropped_times = times[:prefix_size]
                    cropped_times3 = times3[:prefix_size]
                else:
                    cropped_line += prediction_char
                    cropped_times.append(y_time)
                    cropped_times3.append(cropped_times3[-1] + timedelta(seconds=y_t))
                if '!' in cropped_line:
                    continue # make no prediction for this case, since this case has ended already
                ground_truth = ''.join(line[prefix_size:prefix_size+predict_size])
                ground_truth_t = times[prefix_size:prefix_size+predict_size]
                predicted = ''
                predicted_t = []
                enc = encode(cropped_line, cropped_times, cropped_times3)
                #print(enc)
                y = model.predict(enc, verbose=0)
                y_char = y[0][0]
                #print(y_char)
                y_time = y[1][0][0]
                prediction_char , i = getSymbol(y_char)
                #print(i)
                prediction= str(ord(prediction_char)-ascii_offset)
                if y_time<0:
                    y_time=0
                y_t = y_time * divisor
                #print(y_t, y_time)
                predicted_t.append(y_t)
                    
                if prediction == '!': # end of case was just predicted, therefore, stop predicting further into the future
                    print('! predicted, end case')
                    continue
                predicted += prediction
                output = []
                if len(ground_truth)>0:
                    output.append(caseid)
                    output.append(prefix_size)
                    output.append(str(ord(ground_truth)-ascii_offset))
                    output.append(str(predicted))
                    output.append(1 - distance.nlevenshtein(prediction_char, ground_truth))
                    output.append('; '.join(str(x) for x in ground_truth_t))
                    output.append('; '.join(str(x) for x in predicted_t))
                    if len(predicted_t)>len(ground_truth_t): # if predicted more events than length of case, only use needed number of events for time evaluation
                        predicted_t = predicted_t[:len(ground_truth_t)]
                    if len(ground_truth_t)>len(predicted_t): # if predicted less events than length of case, put 0 as placeholder prediction
                        predicted_t.extend(range(len(ground_truth_t)-len(predicted_t)))
                    if len(ground_truth_t)>0 and len(predicted_t)>0:
                        #larger_value = max([ground_truth_t[0]], [predicted_t[0]])
                        output.append(metrics.mean_absolute_error([ground_truth_t[0]], [predicted_t[0]]))
                    else:
                        output.append('')
                        output.append('')
                        output.append('')
                    spamwriter.writerow(output)

In [33]:
make_prediction(lines, case_ids, lines_t, lines_t3)

2023-06-25 19:16:30.375331: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-06-25 19:16:30.645653: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-25 19:16:30.754975: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-25 19:16:30.856597: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-25 19:16:30.865706: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [64]:
result = pd.DataFrame(columns=['Case ID','Prefix length','Groud Truth','Predicted'])

In [65]:
dataframe = df.rename( columns = {'Case ID': 'case:concept:name'})
dataframe = pm4py.format_dataframe(dataframe, case_id='case:concept:name', activity_key='Activity', timestamp_key='Complete Timestamp')
log = pm4py.convert_to_event_log(dataframe)

/var/folders/4w/t_pbdxrn4t5d7m2tkwj7hfrr0000gn/T/ipykernel_10991/3110024940.py:2: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  dataframe = pm4py.format_dataframe(dataframe, case_id='case:concept:name', activity_key='Activity', timestamp_key='Complete Timestamp')
/Users/sumyea/Desktop/Study/Study/WS21-22/Introduction_to_Data_Science/Assignment/Part1/env/lib/python3.8/site-packages/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [67]:
case_ids = []
activities = []
prefix = []
for case in log:
    for idx, event in enumerate(case):
        case_ids.append(case.attributes['concept:name'])
        prefix.append(idx)
        if idx < len(case) - 1:
            activities.append(case[idx+1]['Activity'])
        else:
            activities.append(None)
result['Case ID'] = case_ids
result['Groud truth'] = activities
result['Prefix length'] = prefix
result['Predicted'] = activities

In [60]:
for idx, row in result.iterrows():
    if row['Prefix length'] < 2:
        result = result.drop(idx)

In [68]:
result

,Case ID,Prefix length,Groud Truth,Predicted,Groud truth
0,0,0,0.0,0.0,0.0
1,0,1,0.0,0.0,0.0
2,0,2,14.0,14.0,14.0
3,0,3,14.0,14.0,14.0
4,0,4,17.0,17.0,17.0
...,...,...,...,...,...
4538,98,9,14.0,14.0,14.0
4539,98,10,NaN,NaN,NaN
4540,99,0,14.0,14.0,14.0
4541,99,1,14.0,14.0,14.0


In [69]:
result.to_csv(result_local_path+'result_production.csv')